# Capstone Resumes
## Saxa 4

***

In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from docx import Document
import os
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pdfplumber
import nltk
nltk.download('punkt')
from textblob import TextBlob
import plotly.graph_objects as go
import plotly.express as px
import os
import json
from docx import Document
import pdfplumber

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nicholasreese/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


***

In [9]:
import os
import json
from docx import Document
import pdfplumber

def convert_files_to_json(folder_path, output_json_file):
    corpus = []
    
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Skip temporary files created by Word
        if filename.startswith('~$'):
            print(f"Skipped temporary file: {filename}")
            continue
        
        if filename.endswith('.docx'):
            try:
                doc = Document(file_path)
                text = '\n'.join([para.text for para in doc.paragraphs])
                document_data = {
                    'title': filename,
                    'text': text, 
                    'type': 'word',
                    'file_path': file_path
                }
                corpus.append(document_data)
                print(f"Added .docx: {filename}") 
        
            except Exception as e:
                print(f"Error processing .docx file {filename}: {e}")
        
        elif filename.endswith('.pdf'):
            try:
                with pdfplumber.open(file_path) as pdf:
                    text = '\n'.join([page.extract_text() for page in pdf.pages if page.extract_text()])
                    document_data = {
                        'title': filename, 
                        'text': text,
                        'type': 'pdf',
                        'file_path': file_path
                    }
                    corpus.append(document_data)
                    print(f"Added .pdf: {filename}")
                    
            except Exception as e:
                print(f"Error processing .pdf file {filename}: {e}")
                
    with open(output_json_file, 'w', encoding='utf-8') as json_file:
        json.dump(corpus, json_file, ensure_ascii=False, indent=4)

# Call the function with your paths
convert_files_to_json('/Users/nicholasreese/Desktop/Georgetown/Capstone/NER_Modeling', 'output_corpus.json')
# change your file path to where you saved the resumes

Skipped temporary file: ~$hlyn Bellardine Resume 2024.docx


In [10]:
resumes = pd.read_json('output_corpus.json')

In [7]:
resumes

""


Ashlyn's resume doesnt work because it is not a UTF-8 string. I think its because all of her paragraphs in her resume are in tables or textboxes. I saw something similar when the function did not pick up my table in my resume. 

***

In [5]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

In [6]:
### Converting All Text to lower case

resumes['text'] = resumes['text'].str.lower()

KeyError: 'text'

In [ ]:
resumes

In [ ]:
resumes_values = [f'res{i + 1}' for i in range(len(resumes))]

In [ ]:
resumes.insert(0, 'resume', resumes_values)

In [ ]:
resumes

In [ ]:
vec = CountVectorizer()

In [ ]:
resumes_vec = vec.fit_transform(resumes['text'])

In [ ]:
#print(resumes_vec)

In [ ]:
resumes_vec_dense = pd.DataFrame(resumes_vec.todense(),
                                columns = vec.get_feature_names_out(),
                                index = resumes['resume'])

In [ ]:
print(resumes_vec_dense)

In [ ]:
print(f"Number of Resumes: {resumes_vec_dense.shape[0]}")

In [ ]:
print(f" Number of terms: {resumes_vec_dense.shape[1]}")

***

## Bag of Words for Resumes

In [ ]:
resumes_vec2 = vec.fit_transform(resumes['text'])

In [ ]:
resumes_vec2 = pd.DataFrame(resumes_vec2.todense(),
                           columns = vec.get_feature_names_out())

In [ ]:
resumes_cat = pd.concat([resumes[['resume']], resumes_vec2], axis = 1)

In [ ]:
resumes_cat

***

## TD - IDF

finding the TD-IDF scores of unique words 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tvec = TfidfVectorizer(stop_words = 'english')

In [ ]:
resume_vec = tvec.fit_transform(resumes['text'])

In [ ]:
resumes_tvec = pd.DataFrame(resume_vec.todense(),
                           columns = tvec.get_feature_names_out())

In [ ]:
resumes_tvec

***

## Adding Bag of N-grams

This might help us because we may need word associations that could better filter the pairing of words

In [ ]:
vec_2 = CountVectorizer(ngram_range = (1,2))

In [ ]:
resume_vec2 = vec.fit_transform(resumes['text'])

In [ ]:
resume_vec2 = pd.DataFrame(resume_vec2.todense(),
                          columns = vec.get_feature_names_out())

In [ ]:
resume_vec2

***

## Checking the Most Frequent Words

### One Word Ngrams

In [ ]:
resumes = pd.DataFrame(resumes)
resumes

In [ ]:
vec = CountVectorizer(ngram_range = (1,1), stop_words = 'english')

resume_vec = vec.fit_transform(resumes['text'])
resume_vec = pd.DataFrame(resume_vec.todense(),
                         columns = vec.get_feature_names_out())
resume_df = pd.concat([resumes[['text']], resume_vec],
                     axis = 1)

In [ ]:
word_count = resume_vec.sum(axis = 0)

word_counts_df = pd.DataFrame(word_count, columns = ['count']).reset_index()

word_counts_df.columns = ['word', 'count']

In [ ]:
top_words = word_counts_df.sort_values(by = 'count', ascending = False)
print(top_words)

In [ ]:
top = pd.DataFrame(top_words).nlargest(10, 'count')

fig = go.Figure()

fig.add_trace(go.Bar(
    x=top['word'],
    y=top['count'],
    marker_color='skyblue',
    text=top['count'],
    textposition='auto',  
    hovertemplate='Word: %{x}<br>Count: %{y}<extra></extra>' 
))


fig.update_layout(
    title='Top 10 Most Frequent Resume Words',
    xaxis_title='Words',
    yaxis_title='Count',
    xaxis_tickangle=-45  
)

fig.show()

***

### Two Word Ngrams

In [ ]:
two_word_resume = pd.DataFrame(resumes)

vec = CountVectorizer(ngram_range = (2,2), stop_words = 'english')
two_word_vec = vec.fit_transform(two_word_resume['text'])
two_word_vec = pd.DataFrame(two_word_vec.todense(),
                           columns = vec.get_feature_names_out())

two_word_resume_new = pd.concat([two_word_resume[['text']], two_word_vec],
                               axis = 1)

In [ ]:
word_count_two = two_word_vec.sum(axis = 0)

word_counts_two_df = pd.DataFrame(word_count_two, columns = ['count']).reset_index()

word_counts_two_df.columns = ['word', 'count']

top_two_words = word_counts_two_df.sort_values(by = 'count', ascending = False)
top_two_words

In [ ]:
top = pd.DataFrame(top_two_words).nlargest(10, 'count')

fig = go.Figure()

fig.add_trace(go.Bar(
    x=top['word'],
    y=top['count'],
    marker_color='skyblue',
    text=top['count'],
    textposition='auto',  
    hovertemplate='Word: %{x}<br>Count: %{y}<extra></extra>' 
))


fig.update_layout(
    title='Top 10 Most Frequent Resume Words',
    xaxis_title='Words',
    yaxis_title='Count',
    xaxis_tickangle=-45  
)

fig.show()

***

## Creating a Function for the Words

In [ ]:
def top_words(data,n=10, ngram_range = (1,1)):
    
    data = pd.DataFrame(data)
    
    vec = CountVectorizer(ngram_range = ngram_range, stop_words = 'english')
    data_vec = vec.fit_transform(data['text'])
    vec = pd.DataFrame(data_vec.todense(),
                      columns = vec.get_feature_names_out())
    
    data_new = pd.concat([data[['text']], vec],
                        axis = 1)
    
    word_count = vec.sum(axis = 0)
    word_count_df = pd.DataFrame(word_count, columns = ['count']).reset_index()
    word_count_df.columns = ['word', 'count']
    
    top_words = word_count_df.sort_values(by = 'count', ascending = False)
    top = pd.DataFrame(top_words)
    top = top_words.nlargest(n, 'count')
    
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=top['word'],
        y=top['count'],
        marker_color='skyblue',
        text=top['count'],
        textposition='auto',  
        hovertemplate='Word: %{x}<br>Count: %{y}<extra></extra>' 
))
    
    fig.update_layout(
        title = f'Top {n} Words or Phrases Mentioned in Resumes', #I want to change this in soon
        xaxis_title = 'Words', 
        yaxis_title = 'Count',
        xaxis_tickangle = -45)
    
    fig.show()

In [ ]:
top_words(resumes, 15, ngram_range=(3,3))

## How could we make a function that shows certian key words?

***

## Adding in the Other Resumes

In [11]:
more_resumes = pd.read_csv('Resumes_with_PII_updated.csv')

In [12]:
more_resumes.to_json('more_resumes.json', orient = 'records', lines = True)

In [13]:
more_resumes.rename(columns = {'Resume_str': 'text'}, inplace = True)

In [14]:
more_resumes

,ID,Category,text
0,1,HR,Alexis Lee \n\nalexis.lee@gmail.com\n\n(728) 6...
1,2,HR,Brian Medina \n\nbrian.medina@outlook.com\n\n(...
2,3,HR,Melinda Cohen \n\nmelinda.cohen@gmail.com\n\n(...
3,4,HR,Penny Webb \n\npenny.webb@yahoo.com\n\n(465) 6...
4,5,HR,Jennifer Hansen \n\njennifer.hansen@gmail.com\...
...,...,...,...
2479,2479,AVIATION,Randall Jackson \n\nrandall.jackson@gmail.com\...
2480,2480,AVIATION,Brittany Brown \n\nbrittany.brown@yahoo.com\n\...
2481,2481,AVIATION,Charles Frazier \n\ncharles.frazier@hotmail.co...
2482,2482,AVIATION,Richard Camacho \n\nrichard.camacho@mail.com\n...


In [15]:
more_resumes = pd.DataFrame(more_resumes)
more_resumes

,ID,Category,text
0,1,HR,Alexis Lee \n\nalexis.lee@gmail.com\n\n(728) 6...
1,2,HR,Brian Medina \n\nbrian.medina@outlook.com\n\n(...
2,3,HR,Melinda Cohen \n\nmelinda.cohen@gmail.com\n\n(...
3,4,HR,Penny Webb \n\npenny.webb@yahoo.com\n\n(465) 6...
4,5,HR,Jennifer Hansen \n\njennifer.hansen@gmail.com\...
...,...,...,...
2479,2479,AVIATION,Randall Jackson \n\nrandall.jackson@gmail.com\...
2480,2480,AVIATION,Brittany Brown \n\nbrittany.brown@yahoo.com\n\...
2481,2481,AVIATION,Charles Frazier \n\ncharles.frazier@hotmail.co...
2482,2482,AVIATION,Richard Camacho \n\nrichard.camacho@mail.com\n...


In [17]:
more_resumes.dtypes

ID          object
Category    object
text        object
dtype: object

In [18]:
#more_resumes['text'] = more_resumes['text'].apply(lambda x: x.encode('utf-8').decode('utf-8'))

In [19]:
more_resumes.dtypes

ID          object
Category    object
text        object
dtype: object

In [20]:
#more_resumes.drop(columns = ['Resume_html'], inplace = True)

In [21]:
more_resumes

,ID,Category,text
0,1,HR,Alexis Lee \n\nalexis.lee@gmail.com\n\n(728) 6...
1,2,HR,Brian Medina \n\nbrian.medina@outlook.com\n\n(...
2,3,HR,Melinda Cohen \n\nmelinda.cohen@gmail.com\n\n(...
3,4,HR,Penny Webb \n\npenny.webb@yahoo.com\n\n(465) 6...
4,5,HR,Jennifer Hansen \n\njennifer.hansen@gmail.com\...
...,...,...,...
2479,2479,AVIATION,Randall Jackson \n\nrandall.jackson@gmail.com\...
2480,2480,AVIATION,Brittany Brown \n\nbrittany.brown@yahoo.com\n\...
2481,2481,AVIATION,Charles Frazier \n\ncharles.frazier@hotmail.co...
2482,2482,AVIATION,Richard Camacho \n\nrichard.camacho@mail.com\n...


In [22]:
more_resumes.columns = more_resumes.columns.str.strip()

In [23]:
more_resumes['text'] = more_resumes['text'].apply(lambda x: x.strip() if isinstance(x, str) else x)

In [24]:
more_resumes['text'] = more_resumes['text'].str.lower()

In [25]:
more_resumes

,ID,Category,text
0,1,HR,alexis lee \n\nalexis.lee@gmail.com\n\n(728) 6...
1,2,HR,brian medina \n\nbrian.medina@outlook.com\n\n(...
2,3,HR,melinda cohen \n\nmelinda.cohen@gmail.com\n\n(...
3,4,HR,penny webb \n\npenny.webb@yahoo.com\n\n(465) 6...
4,5,HR,jennifer hansen \n\njennifer.hansen@gmail.com\...
...,...,...,...
2479,2479,AVIATION,randall jackson \n\nrandall.jackson@gmail.com\...
2480,2480,AVIATION,brittany brown \n\nbrittany.brown@yahoo.com\n\...
2481,2481,AVIATION,charles frazier \n\ncharles.frazier@hotmail.co...
2482,2482,AVIATION,richard camacho \n\nrichard.camacho@mail.com\n...


In [26]:
more_resumes['text'] = more_resumes['text'].str.replace('[^\w\s]', '', regex=True)

In [27]:
cats_more_res = more_resumes['Category'].nunique()
cats_more_res

24

In [28]:
cat_counts = more_resumes['Category'].value_counts()
cat_counts

Category
INFORMATION-TECHNOLOGY    120
BUSINESS-DEVELOPMENT      120
FINANCE                   118
ACCOUNTANT                118
ENGINEERING               118
CHEF                      118
AVIATION                  117
ADVOCATE                  117
FITNESS                   117
SALES                     117
BANKING                   115
HEALTHCARE                115
CONSULTANT                115
CONSTRUCTION              112
PUBLIC-RELATIONS          111
HR                        110
DESIGNER                  107
ARTS                      103
TEACHER                   102
APPAREL                    97
DIGITAL-MEDIA              96
AGRICULTURE                63
AUTOMOBILE                 36
BPO                        21
Name: count, dtype: int64

In [29]:
finance = more_resumes[more_resumes['Category'] == 'FINANCE']

In [30]:
engineering = more_resumes[more_resumes['Category'] == 'ENGINEERING']

In [33]:
top_words(engineering, 20, ngram_range = (2,2)) #this is no NER detection

NameError: name 'top_words' is not defined

In [ ]:
top_words(finance, 20, ngram_range = (2,2)) #this is no NER detection

***

### Maybe here this is where we start the NER, although i am guessing this hasnt been added.


In [ ]:
#pip install spacy spacy-transformers

In [ ]:
#import spacy 
#
#nlpspacy_transformer = spacy.load("en_core_web_trf")

In [ ]:
#nlpspacy = spacy.load('en_core_web_md')

In [ ]:
#NER_one = pd.DataFrame({
#    'text': "".join(more_resumes['text'])
#}, index = [0])

In [ ]:
#NER_one

In [ ]:
#wc = WordCloud().generate(' '.join(NER_one['text']))
#plt.imshow(wc)
#plt.show()

In [ ]:
#doc = nlpspacy(NER_one['text'][0])